# Agentic RAG over Thousands of Tables (MELI) — Storytelling Design Doc

> **Goal**: Enable analysts and product teams to ask questions in natural language (ES/PT/EN) and receive fast, governed, and trustworthy answers generated from **thousands of tables** across MELI’s data mesh. The system uses a **planner/orchestrator** and a **multi‑agent skill graph** that safely produces **Abstract Syntax Tree (AST)‑first SQL** and explains lineage, caveats, and cost. 

---

## 1) Executive Summary

- **Problem**: Finding the right tables, joins, and trusted metrics across a large data mesh takes hours and requires deep tribal knowledge.  
- **Solution**: An **agentic, orchestrated RAG‑to‑SQL system** that:  
  1) discovers relevant schemas and join paths,  
  2) drafts and validates SQL via AST + `EXPLAIN`,  
  3) enforces governance (RBAC/ABAC, PII masking, deny lists), and  
  4) returns **answers + charts + SQL + lineage** with consistent SLOs.  
- **Outcomes**: 10× faster time‑to‑insight, reduced rework, governed access by default, unit economics within budget, and complete auditability.

---

## 2) Problem definition - Why now?

- Data volume, teams, and use cases have outpaced traditional BI/SQL assistant patterns.  
- Analysts increasingly **code‑switch** (ES/PT/EN), require **long‑context** retrieval, and need **safe defaults** (time filters, limits).  
- With a central **orchestrator**, we can evolve from a single “SQL assistant” into a **federation of specialist agents** per domain (Ads, Payments, Logistics…).

---

## 3) Narrative Walkthrough (with Diagrams)

### 3.1 Numbered System Overview (arrows 0–9)

The following diagram shows the end‑to‑end data flow, from the user’s NL question to the final response with lineage and visualization.

![Numbered System Overview](Multi-Agentic-RAG.png)

**Step‑by‑step (0–9):**  
0) **User → Orchestrator**: NL question (ES/PT/EN).  
1) **Orchestrator → Schema Searcher**: `RetrieveCandidates` (semantic + graph retrieval).  
2) **Schema Searcher → Orchestrator**: `CandidateSet` (domains/tables/columns, scores, join priors, glossary hits).  
3) **Orchestrator → Join/Plan Synthesizer**: `SynthesizePlan` using CandidateSet + constraints.  
4) **Join/Plan → Orchestrator**: `LogicalPlan` (relations, joins, filters, windows) + confidence.  
5) *(Optional)* **Orchestrator → User**: one **clarifying question** if confidence or required keys are low.  
6) **Orchestrator → Join/Plan**: `UpdatePlan` with user’s answer.  
7) **Orchestrator → SQL Agent**: generate/validate **AST‑first SQL** (style, CTEs) with `EXPLAIN` + AQP.  
8) **SQL Agent → Policy/Guardrails → Executor**: enforce RLS/CLS, masking, deny lists; dry‑run, timeouts, byte‑scan budgets, caching/materializations.  
9) **Answer Composer → User**: **final response** → narrative + chart + SQL + lineage + caveats; observability + evaluator record all steps.

> **Offline services** (catalog, lineage graph, vector index, query‑log index, metric store) power discovery, join priors, and speed via pre‑materialized KPIs.

### 3.2 Hand‑offs in Detail (sequence)

This sequence view shows exactly how the **Orchestrator**, **Schema Searcher**, and **Join/Plan Synthesizer** interact, including the optional clarifier loop.

![Hand‑offs Sequence](Multi-Agents-RAG.png)

---

## 4) Core Design Principles

1) **Planner-first**: a top‑level **orchestrator** routes, retries, and records context.  
2) **AST‑first SQL**: use parsers (`sqlglot`/DB parser) to validate structure and enforce house style.  
3) **Governance by default**: RBAC/ABAC, PII masking, tenancy filters, time‑window defaults, `LIMIT`s, deny lists.  
4) **Cost controls**: `EXPLAIN` → AQP/sample → cache/materialize → full execution; byte‑scan budgets + timeouts.  
5) **Hybrid retrieval**: vector + graph/lineage + query‑log priors; boost freshness, ownership, centrality.  
6) **Multilingual**: ES/PT/EN embeddings and prompts; avoid code‑switching errors via tests and SFT where needed.  
7) **LLMOps rigor**: tracing, golden sets, error taxonomy, self‑repair, and regression tests.

---

## 5) Agents & Interfaces (concise)

### 5.1 Agent Roles
- **Intent & Constraints**: classify task; pull RBAC/ABAC; compute query budget.  
- **Schema Searcher**: retrieve candidate **domains/tables/columns** + join priors.  
- **Join/Plan Synthesizer**: build **LogicalPlan**; ask ≤1 clarifier if required keys are missing.  
- **SQL Drafting Agent**: generate **AST‑first SQL**; `EXPLAIN` cost check; switch to AQP/sample when needed.  
- **Policy/Guardrails**: apply masking, RLS/CLS, tenancy, geo/legal; rewrite SQL if required.  
- **Executor**: dry‑run, timeout, byte caps, cache/materializations.  
- **Answer Composer**: answer + chart + SQL + lineage + caveats; cache by `(user, intent, SQL)`.  
- **Observability & Evaluator**: tool traces; Exec@1/Pass@k; p50/p95; cost/query; FRR; schema‑hallucination rate.

### 5.2 Interface Contracts (I/O)

```json
// Orchestrator → Schema Searcher
Request:  { query_text, query_embedding, constraints, k, freshness_floor }
Response: { candidates:[{domain, table_id, columns, score}], join_priors, glossary_hits, confidence, missing_terms }

// Orchestrator → Join/Plan Synthesizer
Request:  { query_text, candidate_set, user_profile, policies, time_bounds }
Response: { logical_plan(json_ast), required_keys, clarifying_question?, confidence, cost_estimate }
```

---

## 6) Retrieval, Indices & Semantic Layer

- **Schema index** (names, descriptions, tags, owners, row counts, null rates, example values).  
- **Query‑log index** (successful SQL, common **join paths** and filters).  
- **Lineage/graph index** (PK/FK, usage centrality, upstream freshness, data product boundaries).  
- **Metric store / semantic layer** (materialized KPIs by day/week for speed + cost).  
- **Change watcher** (schema deltas → re‑index + agent memory note).

---

## 7) Governance, Safety & Cost

- **Access**: user‑impersonation; RLS/CLS; approvals for cross‑domain joins when required.  
- **Safety**: disallow mutating statements; enforce time bounds and `LIMIT`s by default.  
- **Cost**: timeouts; byte‑scan budgets; AQP previews; materialized KPIs; caching of hot joins/filters.  
- **Auditability**: complete lineage; saved SQL; tool traces; reproducible snapshots.

---

## 8) Quality & Evaluation (LLMOps)

- **Golden sets** per domain: (question, gold SQL, expected subset) for **Exec@1/Pass@3/Result@tol**.  
- **Metrics**: schema‑hallucination rate; groundedness; p50/p95 latency; cost/query; false refusal rate (FRR).  
- **Loops**: error taxonomy → prompt/program fixes → regression tests; usage analytics to prioritize improvements.

---

## 9) SLOs & Success Criteria

- **Latency**: p50 < **5s**, p95 < **20s** for metric‑store questions; p95 < **45s** for cold multi‑join queries.  
- **Effectiveness**: Exec@1 ≥ **80%**, Pass@3 ≥ **95%** in priority domains by week 12.  
- **Safety**: **0** PII leaks; 100% answers carry lineage + SQL + caveats.  
- **Adoption**: ≥ **200** weekly active analysts; ≥ **30%** of questions resolved without SME escalation.  
- **Economics**: cost/query within budget; cache/materialization hit rate tracked and rising.

---

## 10) Team & Infrastructure

- **Platform**: Orchestrator (e.g., Verdi or LangGraph runner); Vector DB; lineage/graph from catalog + logs; sqlglot/DB parser; EXPLAIN hooks; AQP; auto‑viz.  
- **Team pods**:  
  - *AI Platform*: orchestrator, agents, evaluation, guardrails.  
  - *Data Platform*: catalog/lineage, semantic layer, caches/materializations.  
  - *Domain SMEs*: golden sets, join priors, glossary.  
  - *Sec/Compliance*: RBAC/ABAC, PII policies, approvals workflow.  
- **Runbooks**: on‑call, incident taxonomy, safe sandboxes, schema change SLOs.

---

## 11) Risks & Mitigations

- **Schema drift** → automated re‑index + contract tests; agent memory updates.  
- **Low planning confidence** → single clarifier; broaden retrieval; fall back to metric store.  
- **Cost spikes** → EXPLAIN gate, AQP previews, byte caps, cache/materialize, kill‑switches.  
- **Safety gaps** → policy rewrites and denials by default; mandatory lineage + SQL exposure.

---

## 12) Technical Considerations

### 12.1 Orchestration, State, and Reliability
- **Deterministic state machine**: Run the orchestrator as a graph/state machine with an explicit state schema `{ intent, constraints, candidate_set_id, logical_plan_version, policy_decision, budget }`. Persist idempotent step tokens to enable retries and deduplication.
- **Budgets and step guards**: Enforce per‑request ceilings on LLM calls, tokens, and wall‑clock. Track budget consumption in state; degrade gracefully when exceeded.
- **Event backbone**: Emit normalized events (`QueryReceived`, `CandidatesRetrieved`, `PlanSynthesized`, `PolicyDecision`, `Executed`, `AnswerComposed`) to a message bus for observability and async fan‑out (evaluation, analytics).
- **Plan versioning**: Store `logical_plan` as dialect‑aware relational algebra JSON with `plan_version` and a `constraints_digest` to support reproducibility and cache keys.

### 12.2 Retrieval, Indices, and Ranking
- **Hybrid retrieval upgrades**: Combine multilingual dense embeddings (e.g., mE5/LabSE) with BM25/uniCOIL; apply a cross‑encoder re‑ranker over top‑k.
- **Code‑switch normalization**: Language ID, lemmatization, and tri‑lingual glossary synsets for ES/PT/EN prior to embedding.
- **Join‑path discovery**: Build a weighted graph from PK/FK, query logs, lineage centrality, freshness, and ownership trust. Use constrained k‑shortest paths (e.g., Yen’s algorithm) with penalties for cross‑domain hops lacking keys and boosts for observed successful joins.
- **Metadata sanitization**: Treat catalog text as untrusted. Strip HTML/JS, neutralize markdown links, and filter for prompt/metadata injection before LLM use.
- **Index refresh & reproducibility**: CDC‑driven re‑index with per‑entity `schema_version`, soft‑delete handling, blue/green swaps, and request‑scoped `index_snapshot_id`.

### 12.3 Planning and SQL Generation
- **AST‑first enforcement**: Parse with `sqlglot` into a dialect‑aware AST; apply style transforms (CTEs, qualifiers, time filters) programmatically; re‑emit per target dialect.
- **Structural validation**: Verify columns exist, predicates align with graph keys, partition filters are present, and window/frame specs are legal.
- **Cost‑aware planning beyond EXPLAIN**: Pre‑cost with stats (row counts, partitions, NDV, histograms); cap fanout, push down filters, avoid Cartesian joins. If `cost > budget`, transform: tighten time windows, reduce dims, or route to semantic layer.
- **Canonical entity resolution**: Maintain a dictionary for `user_id`, `seller_id`, `order_id` with preferred tables and cross‑domain mappings to reduce ambiguous joins.
- **Decomposition**: Split complex workflows into multi‑query plans (materialize sub‑results) with dependency‑aware execution and partial result caching.

### 12.4 Governance, Safety, and Policy Execution
- **Policy‑as‑code**: Centralize RBAC/ABAC and residency in OPA/Rego or Cedar. The policy agent outputs rewrite clauses and approval requirements.
- **Enforce at source**: Prefer DB‑native RLS/CLS, masking, and row filters over app‑side enforcement for trust‑critical joins.
- **Explainable decisions**: Include reasons and actionable rewrite hints; attach a `policy_decision_id` to traces and cache keys.
- **Injection defenses**: Segment LLM context for user vs metadata; disallow tool‑invoking instructions from retrieved text; use function‑calling schemas to constrain outputs.

### 12.5 Execution, Caching, and Cost Control
- **Cache keys**: Use normalized AST + `dialect` + `policy_decision_id` + normalized time windows for result caches. Hash CTE subtrees for intermediate caches reusable across users when policy‑compliant.
- **AQP strategy**: Apply stratified/date partition sampling or reservoir sampling on high‑cardinality dims; provide confidence intervals; use weighted sampling to handle skew. Prefer DB‑native approximate funcs or run AQP in DuckDB over columnar extracts.
- **Materializations**: Registry of precomputed KPIs/heavy joins with freshness SLAs; track hit rate and evict by staleness/cost.
- **Budget enforcement**: Combine EXPLAIN gating, byte‑scan caps, timeouts, and dynamic downshift (semantic layer, reduced dims) with user‑visible caveats.

### 12.6 Observability, Evaluation, and QA
- **Distributed tracing**: OpenTelemetry spans per agent step with attributes: model, tokens, latency, cache hit, `index_snapshot_id`, `plan_version`, `policy_decision_id`.
- **Metrics**: Per‑domain Exec@1/Pass@k, groundedness, schema‑hallucination, p50/p95 latency, cost/query, FRR exported to Prometheus and reviewed weekly.
- **Golden sets & replay**: Curate by domain/language/time windows; nightly replay of gold + recent real queries; bisect failures by stage (retrieval, plan, policy, execution).

### 12.7 Multilingual and Glossary
- **Tri‑lingual glossary**: Terms mapped to columns/metrics/domains; used in re‑ranking and plan validation.
- **Embedding choice**: Single multilingual model to avoid vector space fragmentation; per‑language stopwords/synonyms.
- **Locale normalization**: Handle ES/PT number/date formats in clarifiers and default time filters.

### 12.8 UX and Clarifiers
- **Targeted clarifiers**: Allow up to 2 turns when confidence is low; present top disambiguations with previews (owner, freshness, row count, sample headers).
- **Explainability**: Always show lineage, applied filters, and cost estimate; offer “why these tables?” with join path rationale.

### 12.9 Scale, Capacity, and SLOs
- **Concurrency control**: Token‑bucket per user/org and per‑agent queues with backpressure; degrade to smaller models or semantic layer under load.
- **Capacity planning**: Pre‑warm embeddings/re‑rankers; shard vector indices; use FAISS/HNSW with IVF‑PQ for scale.
- **Hot‑path optimization**: Short‑circuit metric‑store intents via normalized intent → metric templates, bypassing full LLM planning.

### 12.10 Data Quality and Contracts
- **Contracts**: Enforce schema/data contracts on key tables; alert orchestrator on breaking changes and auto‑retrain retrieval weights.
- **Quality signals**: Freshness, null rates, and anomaly scores feed retrieval re‑ranking and plan risk scoring.

### 12.11 Concrete Near‑Term Additions
- **Tech picks**: LangGraph for orchestration; FAISS/HNSW for vector index; `sqlglot` for AST; OPA for policy; OpenTelemetry for tracing; DuckDB for AQP.
- **Cache normalization**: Implement normalized AST hashing; result/CTE caches with TTLs keyed by `index_snapshot_id` and policy changes.
- **Join‑path scorer**: Weighted k‑shortest paths using query‑log features and ownership; evaluate via ablations in the harness.
- **Prompt & schema hardening**: Metadata sanitization and context segmentation; red‑team tests for prompt/metadata injection.
- **Evaluation harness**: Nightly replay and dashboards for Exec@1/Pass@3, cost, p95, and schema‑hallucination rate by domain/language.

### 12.12 Interface Extensions (non‑breaking)

```json
// Additional fields carried through requests for reproducibility & policy tracing
Common: { index_snapshot_id, plan_version, policy_decision_id, constraints_digest }
```
---





### Why we use an AST (vs. string-bashing SQL)
For our SQL agent, “AST-first” means we build and edit the query as a tree, then render to SQL text at the end.
* Validity: Parser guarantees syntactically correct SQL.
* Safety: Easy to block nodes (DELETE, UPDATE) and auto-inject RLS/CLS, LIMIT, time windows.
* Cost control: We can walk the tree to add filters before execution, then EXPLAIN the exact plan.
* Explainability: Each node maps to the plan we show back (joins, filters, aggregates).
* Programmatic edits: Refactors like “add country filter” or “tighten window to last 90 days” are trivial node edits.

#### Tiny example:

Example Query (human view):
```sql
SELECT p.category, SUM(oi.qty * oi.price) AS revenue
FROM orders o
JOIN order_items oi ON oi.order_id = o.id
JOIN products p ON p.id = oi.product_id
WHERE o.country = 'BR'
  AND o.date BETWEEN '2025-04-01' AND '2025-06-30'
GROUP BY p.category;
```
AST (conceptual view):
```
Query
 ├─ Select
 │   ├─ Column(p.category)
 │   └─ Alias(revenue ← Sum(Mul(oi.qty, oi.price)))
 ├─ From(Table(orders AS o))
 ├─ Join(Table(order_items AS oi), On(Eq(oi.order_id, o.id)))
 ├─ Join(Table(products AS p), On(Eq(p.id, oi.product_id)))
 ├─ Where(And(Eq(o.country,'BR'), Between(o.date,'2025-04-01','2025-06-30')))
 └─ GroupBy(p.category)
```
With an AST we can, for example, inject a default LIMIT 1000 or shorten the date range by editing nodes—no brittle string regexes. Then we pretty-print it back to SQL for execution and for the user to review.